In [10]:
import pandas as pd 
from sqlalchemy import create_engine 

In [11]:
crimes_2019 = "Crime2019.csv"
crimes2019_df = pd.read_csv(crimes_2019)


crimes_2018 = "Crimes2018.csv"
crimes2018_df = pd.read_csv(crimes_2018)
crimes2018_df.head()

joined_df = pd.concat([crimes2019_df, crimes2018_df]).dropna()

new_crime_data = joined_df[['ID', 'Case Number', 'Date', 'IUCR','Description','Community Area' ]]


In [12]:
police_stations = "Police_Stations.csv"
police_stations_df = pd.read_csv(police_stations)
stations_updated_df = police_stations_df[["DISTRICT", "DISTRICT NAME","ADDRESS", "WEBSITE"]]

In [19]:
#Create Local Connection
rds_connection_string = "postgres:postgres@localhost:5432/ChicagoCrime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [20]:
engine.table_names()

['chicago_crime_fact', 'police_station']

In [21]:
#Load Dataframe into SQL chicago_crime_fact table
new_crime_data.to_sql(name='chicago_crime_fact', con=engine, if_exists='append', index=False)

In [22]:
#Confirm data has been added by querying the chicago_crime_fact table
pd.read_sql_query('select * from chicago_crime_fact', con=engine).head()

,PRIMARY_KEY,ID,Case Number,Date,IUCR,Description,Community Area
0,1,11936545,JC565127,2019-12-30,0810,OVER $500,3
1,2,11927752,JC554073,2019-12-20,0334,ATTEMPT ARMED - KNIFE / CUTTING INSTRUMENT,25
2,3,11889579,JC507635,2019-01-01,1562,AGGRAVATED CRIMINAL SEXUAL ABUSE,44
3,4,11836284,JC442862,2019-09-21,051A,AGGRAVATED - HANDGUN,25
4,5,11776789,JC371641,2019-07-30,041A,AGGRAVATED - HANDGUN,23


In [24]:
#Load Dataframe into SQL police_station table
stations_updated_df.to_sql(name='police_station', con=engine, if_exists='append', index=False)

In [25]:
#Confirm data has been added by querying the police_station table
pd.read_sql_query('select * from police_station', con=engine).head()

,PRIMARY_KEY,DISTRICT,DISTRICT NAME,ADDRESS,WEBSITE
0,1,Headquarters,Headquarters,3510 S Michigan Ave,http://home.chicagopolice.org
1,2,1,Central,1718 S State St,http://home.chicagopolice.org/community/distri...
2,3,6,Gresham,7808 S Halsted St,http://home.chicagopolice.org/community/distri...
3,4,11,Harrison,3151 W Harrison St,http://home.chicagopolice.org/community/distri...
4,5,16,Jefferson Park,5151 N Milwaukee Ave,http://home.chicagopolice.org/community/distri...
